In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [22]:
#词袋
from nltk.tokenize import TreebankWordTokenizer   #分词
from collections import Counter

sentence = """Keeping up with friends is faster and easier than ever. Share updates and photos, engage with friends and Pages, and stay connected to communities important to you."""
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(sentence.lower())  #分词的结果
'tokens长度：',len(tokens)
bag_of_words = Counter(tokens)
bag_of_words.most_common(5)
TF = bag_of_words['friends']/len(bag_of_words)  #词项频率
TF

#去停用词
import nltk
stopwords = nltk.corpus.stopwords.words('english')
tokens = [i for i in tokens if i not in stopwords]
'去停用词后tokens长度：',len(tokens)
bag_1 = Counter(tokens)
bag_1.most_common(5)

('tokens长度：', 30)

[('and', 4), ('with', 2), ('friends', 2), (',', 2), ('to', 2)]

0.08695652173913043

('去停用词后tokens长度：', 18)

[('friends', 2), (',', 2), ('keeping', 1), ('faster', 1), ('easier', 1)]

In [1]:
#向量化  TF
from nltk.tokenize import TreebankWordTokenizer   #分词
from collections import Counter

tokenizer = TreebankWordTokenizer()
sentence = ['Keeping up with friends is faster and easier than ever. ']
sentence.append('The most important desktop features of Facebook are also available on the app.')
sentence.append('Facebook also helps you keep up with the latest news and current events around the world.')
doc_token = []
for doc in sentence:
    doc_token .append(sorted(tokenizer.tokenize(doc.lower())))
len(doc_token[0]),len(doc_token[1]),len(doc_token[2])
all_doc_tokens = sum(doc_token,[])     #将所有文档合并
len(all_doc_tokens)
lexicon = sorted(set(all_doc_tokens))
len(lexicon)

from collections import OrderedDict
zero_vector = OrderedDict((token,0) for token in lexicon)
zero_vector

import copy 
doc_vectors = []
for doc in sentence:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    for key,value in token_counts.items():
        vec[key] = value/len(lexicon)
    doc_vectors.append(vec)
doc_vectors

[OrderedDict([('.', 0.03125),
              ('also', 0),
              ('and', 0.03125),
              ('app', 0),
              ('are', 0),
              ('around', 0),
              ('available', 0),
              ('current', 0),
              ('desktop', 0),
              ('easier', 0.03125),
              ('events', 0),
              ('ever', 0.03125),
              ('facebook', 0),
              ('faster', 0.03125),
              ('features', 0),
              ('friends', 0.03125),
              ('helps', 0),
              ('important', 0),
              ('is', 0.03125),
              ('keep', 0),
              ('keeping', 0.03125),
              ('latest', 0),
              ('most', 0),
              ('news', 0),
              ('of', 0),
              ('on', 0),
              ('than', 0.03125),
              ('the', 0),
              ('up', 0.03125),
              ('with', 0.03125),
              ('world', 0),
              ('you', 0)]),
 OrderedDict([('.', 0.03125),
            

In [4]:
# 向量化  TF-IDF
document_tfidf_vectors = []
for doc in sentence:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_count = Counter(tokens)
    for key,value in token_count.items():
        docs_containing_key = 0
        for _doc in sentence:
            if key in _doc:
                docs_containing_key += 1
        tf = value/len(lexicon)
        if docs_containing_key:
            idf = len(sentence)/docs_containing_key
        else:
            idf = 0
        vec[key] = tf * idf
    document_tfidf_vectors.append(vec)
document_tfidf_vectors

[OrderedDict([('.', 0.03125),
              ('also', 0),
              ('and', 0.046875),
              ('app', 0),
              ('are', 0),
              ('around', 0),
              ('available', 0),
              ('current', 0),
              ('desktop', 0),
              ('easier', 0.09375),
              ('events', 0),
              ('ever', 0.09375),
              ('facebook', 0),
              ('faster', 0.09375),
              ('features', 0),
              ('friends', 0.09375),
              ('helps', 0),
              ('important', 0),
              ('is', 0.09375),
              ('keep', 0),
              ('keeping', 0.0),
              ('latest', 0),
              ('most', 0),
              ('news', 0),
              ('of', 0),
              ('on', 0),
              ('than', 0.09375),
              ('the', 0),
              ('up', 0.046875),
              ('with', 0.046875),
              ('world', 0),
              ('you', 0)]),
 OrderedDict([('.', 0.03125),
             

In [8]:
import numpy as np

query = 'Contact with your friends'
query_vec = copy.copy(zero_vector)
tokens = tokenizer.tokenize(query.lower())
token_count = Counter(tokens)

for key,value in token_count.items():
    docs_containing_key = 0
    for doc_ in sentence:
        if key in doc_:
            docs_containing_key += 1
    if docs_containing_key == 0:
        continue
    tf = value/len(tokens)
    idf = len(sentence)/docs_containing_key
    query_vec[key] = tf * idf

def cosine_sim(x,x1):
    x = [i for i in x.values()]
    x1 = [i for i in x1.values()]
    a ,a1,a2= 0,0,0
    for i in range(len(x)):
        a += x[i]*x1[i]
        a1 += x[i]**2
        a2 += x1[i]**2
    return a/(a1**(1/2)*a2**(1/2))
cosine_sim(query_vec,document_tfidf_vectors[0])
cosine_sim(query_vec,document_tfidf_vectors[1])
cosine_sim(query_vec,document_tfidf_vectors[2])


0.42683279491835413

0.0

0.06708203932499368

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentence = ['Keeping up with friends is faster and easier than ever. ']
sentence.append('The most important desktop features of Facebook are also available on the app.')
sentence.append('Facebook also helps you keep up with the latest news and current events around the world.')
corpus = sentence
Vectorizer = TfidfVectorizer(min_df=1)
model = Vectorizer.fit_transform(corpus)
model.todense().round(2)

array([[0.  , 0.26, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.34, 0.  , 0.34,
        0.  , 0.34, 0.  , 0.34, 0.  , 0.  , 0.34, 0.  , 0.34, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.34, 0.  , 0.26, 0.26, 0.  , 0.  ],
       [0.22, 0.  , 0.28, 0.28, 0.  , 0.28, 0.  , 0.28, 0.  , 0.  , 0.  ,
        0.22, 0.  , 0.28, 0.  , 0.  , 0.28, 0.  , 0.  , 0.  , 0.  , 0.28,
        0.  , 0.28, 0.28, 0.  , 0.43, 0.  , 0.  , 0.  , 0.  ],
       [0.2 , 0.2 , 0.  , 0.  , 0.27, 0.  , 0.27, 0.  , 0.  , 0.27, 0.  ,
        0.2 , 0.  , 0.  , 0.  , 0.27, 0.  , 0.  , 0.27, 0.  , 0.27, 0.  ,
        0.27, 0.  , 0.  , 0.  , 0.4 , 0.2 , 0.2 , 0.27, 0.27]])

In [6]:
#齐普夫定律：在给定的自然语言语料库中，任何一个词的频率与它在频率表中的排名成反比
from nltk.corpus import brown
from collections import Counter
import string

# brown.words()[:10]
brown.tagged_words()[:5]
puncs = string.punctuation
puncs += '``'
puncs += '\'\''
word_list = [x.lower() for x in brown.words() if x not in puncs]
tokens_count = Counter(word_list)
tokens_count.most_common(20)

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL')]

[('the', 69971),
 ('of', 36412),
 ('and', 28853),
 ('to', 26158),
 ('a', 23195),
 ('in', 21337),
 ('that', 10594),
 ('is', 10109),
 ('was', 9815),
 ('he', 9548),
 ('for', 9489),
 ('it', 8760),
 ('with', 7289),
 ('as', 7253),
 ('his', 6996),
 ('on', 6741),
 ('be', 6377),
 ('at', 5372),
 ('by', 5306),
 ('i', 5164)]